# FEP Tutorial

The bash script created for FEP uses ambertools and sire. The setup folder has following directory structure and lets call this FEP directory as "home" directory

In [ ]:
!mkdir Ligands Protein Parameters Complex Equilibrium Perturbations

Protein prepared for amber system and ready for amber simulation should be placed in Protein folder as protein2.pdb

All the ligands with name LigXX.mol2 should be placed in Ligand folder

All the parameters used are found in Parameter folder

Perturbations should be defined in "pertlist" file in home directory

FEP simulation scripts are subdivided here into the following steps.
<br>
<font color=green>Parametrisation</font>
<br>- Protein preparation
<br>- Ligand parameterisation
<br>
<font color=green>Unbound state</font>
<br>- Ligand solvation and equilibration
<br>- Setting Alchemical Transformations - Ligand
<br>- Running unbound state lambda simulations
<br>
<font color=green>Bound state</font>
<br>- Complex solvation and equilibration
<br>- Settingup Alchemical Transformations - complex
<br>- Running Bound state lambda simulations
<br>
<font color=green>Analysis</font>
<br>- Visual inspection of trajectories
<br>- Binding_Free_Energy analysis


### <font color=green>**Parameterisation**</font>

<font color=blue>**1_Protein Preparation**</font>

script to use <font color=brown>./1_Protein-prep.sh</font>

a high resolution Ligand bound co-crystal structure is essential for FEP calculation.

protein should be prepared to fix any missing atoms, correct protonation states and cap the termini. This can be done by schrodinger maestro or by using webservers or with different modules in ambertools.

if using schrodinger`s maestro remove all non polar hydrogens and fix residue name of cap NMA to NME

protein2.pdb should be amber atomtyped any errors during protein preparation by tleap should be fixed with carefull inspection. Normally for file prepared by meastro Histidines  gives an error change corresponding residue name from HIS to HID or HIP


<font color=blue>**2_Ligand Parameterisation**</font>

script to use <font color=brown>./2_Lig_param.sh</font>

All ligand should be aligned to the co-crystal structure. A docking method or manipulation of co-crystal structure followed by refinement method should be used for FEP


Ligand input file format should be mol2 or pdb format (but it should be changed in the script). 
<font color=brown>the charge state should be corrected</font> in the script at the line stating "-nc +1"
for each ligand in Ligand folder a subdirectory will be created with ligand name



using antechanber ligands are optimised using AM1 method

### <font color=green>**Solvation and equilibration**</font>

<font color=blue>**3_Ligand solvation and equilibration run**</font>

Script to use <font color=brown>./3_Lig_Sov_eq.sh</font>  for ligands

First a ligand forcefield library file was created then solvated using TIP3P water model. The solvated system is minimised using protocol described in "Parameter/Minimis.cfg" file and then heated to 300 C using "Parameter/Eq1_nvt.cfg" followed by pressure equilibration using "Parameter/Eq2_npt.cfg" and then by a short pre-production run using configuration "Parameter/Eq3_pre.cfg"

A copy for equilibrated system is then placed in Equilibrium folder as  LigXX_Eq.rst7 LigXX_Eq.prm7

<font color=blue>**4_Complex solvation and equilibration run**</font>

Script to use <font color=brown>./4_Com_Sov_eq.sh</font> for protein-ligand complex

This script creates first residue as ligand followed by protein

based on overall charge state of the system counter ions should be changed in the script at the line "addIons2 complex Na+ 0"

The solvated complex has an additional step of minimising water first followed by whole system minimisation, heating, pressure equilibration and short preproduction run

- water minimisation : Watermi.cfg, restrained heavy atoms of solute
- system minimisation: Minimis.cfg, unrestrained
- heating: Eq1_nvt.cfg, NVT, restrained protein backbone heavy atoms temperature raised to 300K in 10ps
- density: Eq2_npt.cfg, NPT, restrained protein backbone heavy atoms pressure equilibrated to 1 atm in 50ps
- pre-production: Eq3_pre.cfg, NPT, unrestrained, 50ps


The above protocol runs on MPI version of sander, if MPI version is not compiled on your machine and to use serial version of sander replace <font color=brown>mpirun -np 6 sander.MPI</font> with <font color=brown>sander</font> in the script

A copy of the equilibrated system is then placed in Equilibrium folder as Pro_LigXX_Eq.rst7 .prm7

### <font color=green>**Setting up and running Alchemical Transformations**</font>

<font color=blue>**5_Settingup Alchemical Transformations for unbound form**</font>

script to use <font color=brown>./5_Ligand_perturbation.sh</font> 

based on the transformation given in the "pertlist" file in parent directory a setup folder is created in Perturbation folder as

LigXX_to_LigYY/unbound

<font color=blue>**6_Settingup Alchemical Transformations for bound form**</font>

script to use <font color=brown>./6_Complex_perturbation.sh</font> 

based on the transformation given in the "pertlist" file in parent directory a setup folder is created in Perturbation folder as

LigXX_to_LigYY/bound

<font color=blue>**7_Running Transformations - Unbound**</font>

script to use <font color=brown>./7_ligand_lambdarun.sh</font> 

Runs in "LigXX_to_LigYY/unbound" folder creates and runs simulation at each lambda specified. Simulation configurations can be changed in the parameter file "Parameters/lambda.cfg"

Runs on openMM with GPU mode

<font color=blue>**8_Running Transformations - Bound**</font>

Script to use <font color=brown>./8_complex_lambdarun.sh</font> 

Runs in "LigXX_to_LigYY/bound" folder creates and runs simulation at each lambda specified. simulation configuration can be changed in the parameter file "Parameters/lambda.cfg"

Runs on openMM with GPU mode

### <font color=green>**Analysis**</font>

<font color=blue>**Visual inspection of trajectories**</font>

Jupyter notebook to use <font color=brown>Visualising_trajectories.ipynb</font> 

Needs pytraj module from ambertools for trajectory analysis, output is written as aligned.pdb

To set environment use this on my system <br>
source /home/nvk28/Desktop/pytraj.sh

<font color=blue>**Binding_Free_Energy analysis**</font>

Jupyter notebook to use <font color=brown>Binding_Free_Energy.ipynb</font> 

needs analyse_freenrg module from sire for relative binding energy calculation. this file is found inside "Perturbations/LigXX_to_LigYY" folder

to set environment use this on my system 
<br>
source /home/nvk28/Desktop/fep.sh